In [1]:
import torch as t

In [7]:
t.nn.functional.conv_transpose1d(t.tensor([[[1,1,1]]]), t.tensor([[[1,1,1]]]), stride=2)

tensor([[[1, 1, 2, 1, 2, 1, 1]]])

In [3]:
from unet import Text2Im
from reference.glide_text2im.tokenizer.bpe import get_encoder

In [21]:
tokenizer = get_encoder()
model = Text2Im(tokenizer)

In [25]:
len(model.state_dict().keys())

709

In [27]:
import sys

sys.path.append("./reference")
import torch as th

from reference.glide_text2im.download import load_checkpoint
from reference.glide_text2im.model_creation import (
    create_model_and_diffusion,
    model_and_diffusion_defaults,
)

device = th.device('cpu')
options = model_and_diffusion_defaults()
options['timestep_respacing'] = '100'
ref_model, diffusion = create_model_and_diffusion(**options)
ref_model.load_state_dict(load_checkpoint('base', device))

TypeError: ResBlock.__init__() got multiple values for argument 'out_channels'